In [1]:
import pyspark
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.sql.functions import max
from pyspark.sql.functions import col
from pyspark.sql.types import StructType,StructField, StringType

# Creating the SparkSession
spark = SparkSession.builder.appName('Case_Study_Adidas').master("local").getOrCreate()

# 1. Load the data

In [2]:
# Load JSON file into dataframe
path = 'C:\\BigData\\ol_cdump.json'
data_df = spark.read.option("multiline", "false").json(path)

# print(type(df))
# Printing the Schema
# df.printSchema()

# 2. Make sure your data set is cleaned enough, so we for example don't include in results with empty/null "titles".

In [3]:
# Filtering the data where the title is null

df = data_df.filter(data_df.title.isNotNull())
df.select("title","publish_date").show(truncate=False)

+-----------------------------------------------------------------------------------------------------+------------+
|title                                                                                                |publish_date|
+-----------------------------------------------------------------------------------------------------+------------+
|The effect of differentiated marking tools and motivational treatment on figural creativity          |1975        |
|Comparison of the nominal grouping and sequenced brainstorming techniques of creative idea generation|1976        |
|Professional accident investigation                                                                  |1977        |
|I chauceriani scozzesi                                                                               |1964        |
|Lezioni zurighesi sul Petrarca                                                                       |1955        |
|El saber ginecologico del padre Feijoo                         

# 3.1 Select all "Harry Potter" books

In [4]:
# Select all "Harry Potter" books

df.filter(col("title").contains("Harry Potter")).select("title","publish_date").show(truncate=False)

+----------------------------------------+-------------+
|title                                   |publish_date |
+----------------------------------------+-------------+
|Harry Potter and the philosopher's stone|1998         |
|Harry Potter y la piedra filosofal      |2000         |
|The Science of Harry Potter             |June 23, 2003|
+----------------------------------------+-------------+



# 3.2 Get the book with the most pages

In [5]:
# To Get the book with the most pages

df.createOrReplaceTempView("records")
spark.sql("select number_of_pages,title,publish_date from(select *, dense_rank() over(order by number_of_pages desc)r from records) where r=1").show(truncate=False)

+---------------+-----------------------------+------------+
|number_of_pages|title                        |publish_date|
+---------------+-----------------------------+------------+
|48418          |Nihon shokuminchi kenchikuron|2008        |
+---------------+-----------------------------+------------+



In [6]:
# To Get the book with the most pages(No of Pages)

df.groupby().max('number_of_pages').collect()[0].asDict()['max(number_of_pages)']

48418

# 3.3 Find the Top 5 authors with most written books (assuming author in first position in the array, "key" field and each row is a different book)

In [7]:
# Find the Top 5 authors with most written books 
# (assuming author in first position in the array, "key" field and each row is a different book)

sql_df =spark.sql("select authors from(select *, dense_rank() over(order by number_of_pages desc)r from records) where r between 1 and 5")
print(sql_df.show(truncate=False))

+-----------------------------------+
|authors                            |
+-----------------------------------+
|[{null, /authors/OL5510271A, null}]|
|[{null, /authors/OL4442921A, null}]|
|[{null, /authors/OL607566A, null}] |
|[{null, /authors/OL130993A, null}] |
|[{null, /authors/OL726653A, null}] |
+-----------------------------------+

None


# 3.4 Find the Top 5 genres with most books

In [8]:
# Find the Top 5 genres with most books

df_geners = df.filter(data_df.genres.isNotNull())
df_geners.select("genres").show(truncate=False)

+------------------------------------+
|genres                              |
+------------------------------------+
|[Early works to 1800]               |
|[Outlines, syllabi, etc]            |
|[Bibliography]                      |
|[Conversation and phrase books]     |
|[Texts]                             |
|[Bibliography]                      |
|[Readers]                           |
|[Texts]                             |
|[Drama]                             |
|[Bibliography]                      |
|[Folklore]                          |
|[Bibliography]                      |
|[Juvenile literature, Bibliography.]|
|[Bibliography]                      |
|[Librettos]                         |
|[Dictionaries]                      |
|[Juvenile literature]               |
|[Handbooks, manuals, etc]           |
|[Sources]                           |
|[Bibliography, Catalogs]            |
+------------------------------------+
only showing top 20 rows



In [ ]:

df_geners.groupby().max('genres').collect()[0].asDict()['max(genres)']

# 3.5 Get the avg. number of pages

In [9]:
# Get the avg. number of pages
df.groupby().avg('number_of_pages').collect()[0].asDict()['avg(number_of_pages)']

222.65999078989668

In [ ]:
# Per publish year, get the number of authors that published at least one book
